# Seminar Quantitativ Microbiology 
# Metabolite Production with Strain Engineering

## Introduction


For a overview on Jupyter notebooks read [this review](https://www.nature.com/articles/d41586-018-07196-1). 

---

## Tutorial Steps
  * Set up of Python environment
    * Basic libraries(sys, pandas, numpy, matplotlib, zipfile, cobrapy)
  * Analysis of Genome Scale Metabolic Model
    * Retrieval of GSMM for *P. pastoris*
    * Flux variability of exchange reactions
    * Minimal medium composition
  * Experimental growth rate reproduction
    * Familiarizing with biomass composition reactions
    * Defining functions for correct biomass equation switch
    * Data retrieval
    * Simulation loop
    * Graphical output


## Set-up compute environment

Before we can analyse GSMM, we have adjust the python environment that it integrates the cobrapy toolbox and downloading the GSMM.

### Basic Python libraries 
Some libraries that facilitate data manipulation

In [1]:
import sys # loading commands to control/navigate within the system architecture
# Loading pandas, a library for data manipulation
from os.path import join
# import xlrd
import pandas as pd
# import lxml

# Loading numpy, a library fo manipulation of numbers
import numpy as np

# loading matplotlib, a library for visualization
import matplotlib.pyplot as plt
%matplotlib inline

# loading cobrapy, a library dedicated to the analysis of genome scale metabolic models
from cobra.io import read_sbml_model, write_sbml_model, load_matlab_model

# loading escher for metabolic network visualization
import escher
from escher import Builder
# from time import sleep
# escher.rc['never_ask_before_quit'] = True
# list of available maps
# print(escher.list_available_maps())

# loading Memote, quality assessment of GSMM
# from memote import test_model, snapshot_report

from biolabsim import Host, Strain, Ecol

print('System ready')

System ready


In [2]:
wtHost = Ecol()
wtHost.show_BiotechSetting()
print('Wild type growth rate: {:.2f}'.format(wtHost.strain.objective))
print()
myHost = wtHost.clone_with_mutation('Mut1')
myHost.show_BiotechSetting()
print('Mutation growth rate: {:.2f}'.format(myHost.strain.objective))

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Host: Ecol
Resources: 40
Substrate: None
Strain: WT
Wild type growth rate: 0.87



/home/ulf/anaconda3/envs/biolabsim/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/ulf/anaconda3/envs/biolabsim/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.22.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/ulf/anaconda3/envs/biolabsim/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/ulf/anaconda3/envs/biolabsim/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestR

resetting boundaries
Host: Ecol
Resources: 40
Substrate: None
Strain: Mut1
Mutation growth rate: 0.49


In [3]:
from biolabsim.measurement.sequencing import measure_BaseCompare
from biolabsim.measurement.fluxes import measure_EnzymeLevel1
myDiff = measure_BaseCompare(wtHost.strain.genome, myHost.strain.genome)
a,b,c = measure_EnzymeLevel1(wtHost.name, wtHost.strain, myHost.strain)
print(myDiff)
print(b,c)

[[1761, ('T', 'A')], [1765, ('C', 'T')], [3251, ('A', 'G')], [3253, ('T', 'G')], [4354, ('C', 'G')], [4355, ('T', 'A')]]
{'lower': array([29]), 'upper': array([54, 72])} {'increase': array([], dtype=int64), 'decrease': array([54, 72, 29])}


In [4]:
mylist = np.array(wtHost.strain.genes_df['Expression'])
mylist2 = np.array(myHost.strain.genes_df['Expression'])
print('Gene expression differences')
np.round(mylist2/mylist,2)

Gene expression differences


array([1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.92, 1.  , 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.56,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.95, 1.  , 1.  , 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ,
       1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ])

In [5]:
myl = np.array(wtHost.strain.genes_df['Expr2Flux'])
myl2 = np.array(myHost.strain.genes_df['Expr2Flux'])
# myHost.strain.genes_df
print('Exp2Flux diff')
np.round(myl2/myl,2)

Exp2Flux diff


<ipython-input-5-793ddc7a46d2>:5: RuntimeWarning: invalid value encountered in true_divide
  np.round(myl2/myl,2)


array([ 1., nan,  1.,  1.,  1., nan, nan,  1.,  1., nan, nan, nan,  1.,
        1.,  1.,  1., nan, nan, nan, nan,  1.,  1., nan,  1.,  1., nan,
        1.,  1.,  1.,  1., nan,  1., nan,  1., nan, nan,  1.,  1.,  1.,
       nan,  1.,  1.,  1., nan, nan, nan,  1., nan, nan, nan, nan,  1.,
       nan, nan,  1.,  1., nan, nan,  1.,  1.,  1., nan, nan,  1., nan,
       nan, nan, nan, nan,  1., nan,  1.,  1.,  1.,  1., nan,  1., nan,
       nan, nan,  1.,  1.,  1., nan, nan, nan, nan,  1., nan, nan,  1.,
       nan,  1.,  1.,  1.])

In [6]:
FluxDiff = myHost.strain.genes_df['Fluxes']/wtHost.strain.genes_df['Fluxes']
print('Flux reaction differences')
np.round(np.nan_to_num(FluxDiff, nan=1, posinf=1, neginf=1),2)

Flux reaction differences


array([0.6 , 1.  , 0.66, 0.59, 0.51, 1.  , 1.  , 0.6 , 0.56, 1.  , 1.  ,
       1.  , 0.56, 0.65, 0.65, 1.  , 1.  , 1.  , 1.  , 1.  , 0.66, 0.62,
       1.  , 0.73, 0.56, 1.  , 0.62, 0.5 , 0.65, 0.53, 1.  , 0.62, 1.  ,
       0.6 , 1.  , 1.  , 0.66, 0.66, 0.51, 1.  , 0.51, 0.5 , 0.6 , 1.  ,
       1.  , 1.  , 0.62, 1.  , 1.  , 1.  , 1.  , 0.58, 1.  , 1.  , 0.56,
       0.61, 1.  , 1.  , 0.56, 0.62, 0.56, 1.  , 1.  , 0.6 , 1.  , 1.  ,
       1.  , 1.  , 1.  , 0.66, 1.  , 0.51, 0.59, 0.58, 0.56, 1.  , 0.56,
       1.  , 1.  , 1.  , 0.51, 0.61, 0.65, 1.  , 1.  , 1.  , 1.  , 0.66,
       1.  , 1.  , 0.61, 1.  , 0.56, 0.62, 0.62])

In [ ]:
wtHost.Export_Genome()
wtHost.Export_ORFAnnotation()

## Promoter identification with SAPPHIRE

In [ ]:
%load_ext watermark
%watermark -v -m -p ipywidgets,matplotlib,numpy,pandas,sklearn,scipy,joblib,watermark,escher,cobra